In [1]:
from IPython.utils import io
import sys
import pandas as pd
import bs4 as bs
import urllib2
import re
import numpy as np
import uuid
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.image import ImageWriter
from pdfminer.converter import  TextConverter
from pdfminer.pdfpage import PDFPage
reload(sys)
sys.setdefaultencoding('utf-8')
from selenium import webdriver
import time
import glob
import os


In [2]:
class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""


    def __init__(self, stdout='', stderr=''):
        self.stdout = stdout
        self.stderr = stderr

    def __enter__(self):
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr

        if self.stdout:
            sys.stdout = open(self.stdout, 'w')
        if self.stderr:
            if self.stderr == self.stdout:
                sys.stderr = sys.stdout
            else:
                sys.stderr = open(self.stderr, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr

In [3]:
# to download a copy 
# https://stackoverflow.com/questions/24844729/download-pdf-using-urllib
    

# url of pdf like https://arxiv.org/pdf/1709.00042.pdf
# outputs a list of emails
def getEmailList(code):

    with redirect_output("txt/" + code + ".txt"):
        %run pdf2txt.py pdf/temp.pdf
    
    with open("txt/" + code + ".txt", 'r') as myfile:
        data=myfile.read()
        emails = re.findall(r'[\w\.-]+@[\w\.-]+', data )
        if emails:
            return emails
        else:
            return []

In [4]:
def download_pdf(lnk, name, download_folder, path_to_chrome_driver):

    options = webdriver.ChromeOptions()
    profile = {
               "plugins.plugins_list": [{"enabled": False,
                                         "name": "Chrome PDF Viewer"}],
               "download.default_directory": download_folder,
               "download.extensions_to_open": ""
                }

    options.add_experimental_option("prefs", profile)
    print("Downloading file from link: {}".format(lnk))
    driver = webdriver.Chrome(path_to_chrome_driver,chrome_options = options)
    driver.get(lnk)

    filename = lnk.split("/")[4].split(".cfm")[0]
    print("File: {}".format(filename))
    print("Status: Download Complete.")
    print("Folder: {}".format(download_folder))
    time.sleep(10)
    
     # get that pdf
    list_of_files = glob.glob('pdf/*')
    latest_file = max(list_of_files, key=os.path.getctime)
    latest_full_path = '/home/ahmed/Dropbox/DFCI/08_radiomics.io/science/ieee/' + latest_file 
    # change its name
    os.rename( latest_full_path, 
              '/home/ahmed/Dropbox/DFCI/08_radiomics.io/science/ieee/pdf/' + name.replace("/","-") + '.pdf')

    driver.close()


In [16]:
df = pd.DataFrame.from_csv('ieee.csv', index_col=0)

print df.shape
df.head()

,Title,Description,ShortDetails,abstract,email,fullURL,source,year,key,use
0,Multi-Scale Rotation-Invariant Convolutional N...,"Q. Wang, Y. Zheng, g. yang, W. Jin, X. Che...",IEEE Journal of Biomedical and Health Informat...,We propose a new Multi-scale Rotation-invarian...,"[shdyn2000@mail.sdu.edu.cn, jie@gmail.com, gpy...",http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2017,b2b029e4-5981-49d6-bc8c-d5bd30eefda6,1
1,Adrenal lesions detection on low-contrast CT i...,"L. Bi, J. Kim, T. Su, M. Fulham, D. Feng, ...",2017 IEEE 14th International Symposium on Biom...,Adrenal lesions include a wide variety of beni...,[],http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2017,afdcc501-bdbd-478c-a307-b8e7073d9ee9,1
2,Anatomy-specific classification of medical ima...,"H. R. Roth, C. T. Lee, H. C. Shin, A. Seff,...",2015 IEEE 12th International Symposium on Biom...,Automated classification of human anatomy is a...,[],http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2015,a0db122c-b66d-4c48-a0c5-3886f3835497,1
3,Deep Convolutional Neural Network for Inverse ...,"K. H. Jin, M. T. McCann, E. Froustey, M. Unser",IEEE Transactions on Image Processing. 2017,"In this paper, we propose a novel deep convolu...",[holger.roth@nih.govorrms],http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2017,3138b542-caa6-4d54-a731-66e3dd97d844,1
4,Convolutional Neural Networks for Medical Imag...,"N. Tajbakhsh, J. Y. Shin, S. R. Gurudu, R. ...",IEEE Transactions on Medical Imaging. 2016,Training a deep convolutional neural network (...,"[tajbakhsh@asu.edu, sejong@asu.edu, gurudu.sur...",http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2016,c0cee720-9517-4fea-a54c-111fe20bd312,1


In [21]:
#  save as much pdfs as possible
for i in range(505,df.shape[0]):
    #
    url =  'http://sci-hub.bz/' + df.iloc[i]['DOI']
    print url
    # downloaf pdf in pdfs folder
    download_pdf(url, df.iloc[i]['DOI'] , 'pdf', '/home/ahmed/Dropbox/DFCI/08_radiomics.io/chromedriver')

    print i

In [6]:
# now most odfs have beenn saved
# now build csv and convert pdf to txt, to get email

In [6]:
def getEmail(pdf):
    file_path =  'pdf/' + pdf
    txt_file = "txt/" + pdf + ".txt"
        
    try:
        with redirect_output(txt_file):
            %run pdf2txt.py $file_path

        with open(txt_file, 'r') as myfile:
            data=myfile.read()
            emails = re.findall(r'[\w\.-]+@[\w\.-]+', data )
            if emails:
                return emails
            else:
                return []
    except:
        return []

In [7]:
# search result page
allTitles = []
allAuthors = []
allFullURLs = []
# abstract page
allEmails = [] 
allYears = []
allAbstracts = []
allComments = [] # combine with year into shortDetails

#added at pandas level
# source = ieee
# key = uuid
# use = all 1


for i in range(df.shape[0]): # 
    #
    allTitles.append(df.iloc[i]['Document Title'])
    # 
    allAuthors.append(df.iloc[i]['Authors'].replace(";",", "))
    #
    allFullURLs.append(df.iloc[i]['PDF Link'])
    # emails
    # check if pdf exists:
    fname =  df.iloc[i]['DOI'].replace("/","-") + '.pdf'
    print fname
    if os.path.isfile('pdf/' + fname):
        allEmails.append(  getEmail ( fname ) )
    else:
        allEmails.append([])
    #
    allYears.append(int(df.iloc[i]['Year']))
    # 
    allAbstracts.append(df.iloc[i]['Abstract'])
    #
    allComments.append(df.iloc[i]['Publication Title'])
    print i
    

PSTypeError: Literal required: <PDFObjRef:48>

PSTypeError: Literal required: /'begin'

PSTypeError: Literal required: <PDFObjRef:69>

PSTypeError: Literal required: /'begin'

PSTypeError: Literal required: /'begin'

...

PDFSyntaxError: Unexpected EOF

PSTypeError: Literal required: /'begin'

PSTypeError: Literal required: /'begin'

PSTypeError: Literal required: /'begin'

PSTypeError: Literal required: /'begin'

In [8]:
lista = [allTitles,allAuthors,allFullURLs,allComments,allYears,allAbstracts,allEmails]
for lis in iter(lista):
    print len(lis)

In [25]:
df = pd.DataFrame()
# those that match pubmed
df['Title'] = allTitles
df['Description'] = allAuthors
df['ShortDetails'] = [allComments[x] + ". " + str(allYears[x]) for x in range(len(allYears))  ]
df['abstract'] =   allAbstracts
df['email'] =   allEmails
df['fullURL'] = allFullURLs                                                                           
df['source'] = ['ieee' for x in range(len(allYears))]                                                                            
df['year'] = allYears
df['key'] = [str(uuid.uuid4()) for x in range(len(allYears))]
df['use'] = [1 for x in range(len(allYears))]
df['doi'] = df1['DOI']
    
    
df.head(n=5)

,Title,Description,ShortDetails,abstract,email,fullURL,source,year,key,use,doi
0,Multi-Scale Rotation-Invariant Convolutional N...,"Q. Wang, Y. Zheng, g. yang, W. Jin, X. Che...",IEEE Journal of Biomedical and Health Informat...,We propose a new Multi-scale Rotation-invarian...,"[shdyn2000@mail.sdu.edu.cn, jie@gmail.com, gpy...",http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2017,fe67cb19-17f9-4b93-9d3b-bc645fc3e4fc,1,10.1109/JBHI.2017.2685586
1,Adrenal lesions detection on low-contrast CT i...,"L. Bi, J. Kim, T. Su, M. Fulham, D. Feng, ...",2017 IEEE 14th International Symposium on Biom...,Adrenal lesions include a wide variety of beni...,[],http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2017,14374988-c785-44ca-acd2-3d2eedb0f862,1,10.1109/ISBI.2017.7950660
2,Anatomy-specific classification of medical ima...,"H. R. Roth, C. T. Lee, H. C. Shin, A. Seff,...",2015 IEEE 12th International Symposium on Biom...,Automated classification of human anatomy is a...,[],http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2015,d1902f1a-4ee3-4830-9570-8edba442116d,1,10.1109/ISBI.2015.7163826
3,Deep Convolutional Neural Network for Inverse ...,"K. H. Jin, M. T. McCann, E. Froustey, M. Unser",IEEE Transactions on Image Processing. 2017,"In this paper, we propose a novel deep convolu...",[holger.roth@nih.govorrms],http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2017,b09a8d9a-f829-4e34-ad97-fe00ac020911,1,10.1109/TIP.2017.2713099
4,Convolutional Neural Networks for Medical Imag...,"N. Tajbakhsh, J. Y. Shin, S. R. Gurudu, R. ...",IEEE Transactions on Medical Imaging. 2016,Training a deep convolutional neural network (...,"[tajbakhsh@asu.edu, sejong@asu.edu, gurudu.sur...",http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,ieee,2016,ec0a06c3-5f4a-4307-a7bc-91ceb4e99152,1,10.1109/TMI.2016.2535302


In [18]:
df.to_json( 'ieee_final.json' ,"records")
# then need to add
# var Data1 = ....
# export default Data1

KeyError: False